In [1]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout,GridspecLayout,interact, interact_manual,IntProgress,VBox,HBox,Button, ButtonStyle, Output, FileUpload
from IPython.display import display, clear_output
import os
import subprocess
import lxml
# need pandas version >= 1.3

# [DataPipe_signalement_docelec] Procédures automatisées de signalement de la documentation électronique

## Lancer un workflow

**Attention : pour le workflow ftf, veiller à bien mettre à jour xslt/tables_conversion.xsl si besoin**

In [3]:
w_options = ['ftf', 'cairn_qsj', 'cairn_titre_a_titre', 'cyberlibris', 'numilog']
atoz_cols_to_remove = ['Edition','Editor', 'Illustrator', 'DOI', 'PeerReviewed','CustomCoverageBegin',
       'CustomCoverageEnd', 'CoverageStatement', 'Embargo', 'CustomEmbargo',
       'Description', 'Subject', 'PackageContentType',
       'CreateCustom', 'HideOnPublicationFinder', 'Delete',
       'OrderedThroughEBSCO', 'IsCustom', 'UserDefinedField1',
       'UserDefinedField2', 'UserDefinedField3', 'UserDefinedField4',
       'UserDefinedField5', 'PackageType', 'AllowEbscoToAddNewTitles',
       'Unnamed: 40']
spin = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    description='Loading:',
    bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
workflow = widgets.Dropdown(
    value='ftf', 
    options=w_options, 
    description='Choisir le workflow',
    style={'description_width': 'initial'},
    layout = Layout(width='350px')
)
filename = widgets.Text( 
    description='Nom du fichier source (déposée dans /source_files/)',
    value='par exemple : atoz_ftf_records.csv',
    style={'description_width': 'initial'},
    layout = Layout(width='600px')
)
tempfiles = widgets.Checkbox(
            value=False,
            description='Supprimer les fichiers temporaires de traitement',
            disabled=False,
        )
button = widgets.Button(description="Start",button_style='primary')
output = widgets.Output()

h1 =  widgets.HBox(
    [
        filename, 
        tempfiles
    ]
)

h2 = widgets.VBox(
    [
        button,
        spin, 
        output
    ]
)

display(workflow,h1,h2)
def file_path(relative_path):
    folder = os.path.dirname(os.path.abspath("__file__"))
    path_parts = relative_path.split("/")
    new_path = os.path.join(folder, *path_parts)
    return new_path

def on_button_clicked(b):
    with output:
        clear_output()
        if workflow.value == "ftf":
            spin.value = 5
            %run execute_workflow.py -w:atoz -f:{filename.value} 
            spin.value = 10
            if str(tempfiles.value) == "True":
                os.system("rm -rf temporary_files/")
                print('temp atoz file deleted')
            '''df = pd.read_csv("source_files/"+filename.value, sep=',', encoding='utf8')
            spin.value = 1
            #clean
            df = df.drop(atoz_cols_to_remove, axis=1).fillna("").replace("&", "&amp;")
            spin.value = 2
            # custom bsc cleaning
            df.drop(df[(df.PackageName == 'Business Source Complete') & ((df.ResourceType == 'Report') | (df.ResourceType == 'Book Series'))].index, inplace=True)
            spin.value = 3
            df.to_xml(path_or_buffer="temporary_files/atoz_export.xml", root_name='Resources', row_name='Resource', encoding='utf-8', xml_declaration=True, pretty_print=True, parser='lxml')
            spin.value = 4
            print(subprocess.run(["run_saxon.bat",file_path("xslt/atoztemp4primo.xsl"),file_path("temporary_files/atoz_export.xml"),file_path("result_files/atoz.xml")], shell=True, check=True, capture_output=True))
            spin.value = 5
            '''
            
        if workflow.value == "numilog":
            spin.value = 5
            %run execute_workflow.py -w:numilog -f:{filename.value}
            spin.value = 10

button.on_click(on_button_clicked) 

## Résultat

Le fichier résultat en xml (portant le nom du workflow utilisé) est récupérable dans DataPipe_signalement_docelec/result_files/